In [1]:
import os
import pandas as pd
from pandas import Series, DataFrame

Data sources:
- https://ttu-ir.tdl.org/bitstream/handle/2346/74082/ICES_2018_81.pdf

In [2]:
## Get current working directory
print("Current working directory:", os.getcwd())

## Find path to raw data (for the raw files)
raw_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'raw_data'))

## Find path to folder for saving cleaned csv
cleaned_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'cleaned_data'))


## Find path to folder for inflation
inflation_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'inflation'))

Current working directory: /Users/jennagreene/Documents/GitHub/HATCH_data/reading_files


In [3]:
target_inflation_file = "/A001RG3A086NBEA.xls"
target_inflation_filepath = inflation_data_path + target_inflation_file

target_file = '/cost data/space launches.xlsx'
target_filepath = raw_data_path + target_file

In [4]:
## adjusting for inflation from 2018 USD to 2022 USD
nipa = pd.read_excel(target_inflation_filepath, header=10)

year_list = []
for x in nipa['observation_date']:
    x = int(str(x)[:4])
    year_list.append(x)
nipa['Year'] = year_list
nipa.set_index('Year', drop=True, inplace=True)
nipa.drop(columns='observation_date', inplace=True)
nipa = nipa.transpose()

infl_2018_2022 = float(nipa[2022]/nipa[2018])
infl_2018_2022

/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_30938/438783119.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  infl_2018_2022 = float(nipa[2022]/nipa[2018])


1.1528881798983042

In [5]:
space_launches = pd.read_excel(target_filepath, header=1, usecols=[1,2], skipfooter=1)
space_launches = space_launches.groupby(['First launch date']).mean()
space_launches = space_launches.transpose()
space_launches.iloc[0] = space_launches.iloc[0] * infl_2018_2022
space_launches['Data Source'] = 'NASA'
space_launches['Spatial Scale'] = 'Global'
space_launches['Country Code'] = 'World'
space_launches['Country Name'] = 'World'
space_launches['Metric'] = 'Price'
space_launches['Unit'] = 'Thousand 2022 USD/kg'
space_launches['Technology Name'] = 'Space Launches'
space_launches['ID'] = space_launches['Technology Name'] + '_' + space_launches['Metric'] + '_' + space_launches['Country Code']
space_launches.set_index('ID', drop=True, inplace=True)
space_launches.columns.name = None
space_launches

,1957,1960,1961,1962,1964,1965,1966,1967,1968,1971,...,2010,2012,2018,Data Source,Spatial Scale,Country Code,Country Name,Metric,Unit,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Space Launches_Price_World,1031.489055,193.454637,128.892899,35.739534,32.280869,4.726842,14.353458,11.759459,5.995019,8.877239,...,3.112798,11.528882,1.614043,NASA,Global,World,World,Price,Thousand 2022 USD/kg,Space Launches


In [6]:
output_file = 'nasa_prices.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)

space_launches.to_csv(output_file_path)
print("Data saved to:", output_file_path)


Data saved to: /Users/jennagreene/Documents/GitHub/HATCH_data/cleaned_data/nasa_prices.csv
